In [ ]:
from matplotlib import pyplot as plt
import tensorflow as tf
from tensorflow.python.data import AUTOTUNE

from finalproject.data_loading import process_path
from finalproject.network import Net
from finalproject.loss import from_grid_to_coordinates
import PIL.Image
from finalproject.utils import annotate_image

grid_h = 1
grid_w = 1
batch_size = 1
epochs = 20

list_ds = tf.data.Dataset.list_files('datasets_yolo/train/*.jpg', shuffle=False)
train_ds = list_ds.map(process_path, num_parallel_calls=AUTOTUNE)
train_ds = train_ds.batch(batch_size).prefetch(buffer_size=AUTOTUNE)

model = Net()
model.build((None, 224, 224, 3))
model.load_weights('saved_model/my_checkpoint.weights.h5')

for images, labels, file_path in train_ds:
    preds = model.predict(images)


    prediction = preds[0]

    prediction = from_grid_to_coordinates(prediction)
    img = PIL.Image.open(file_path[0].numpy().decode('utf-8'))
    img = annotate_image(img, targets=prediction[:, 0:4], scores=prediction[:, 4], labels=tf.constant([0]), normalized=True)
    plt.figure(figsize=(4, 4), facecolor='black')
    plt.imshow(img)
    plt.axis('off')
    plt.show()



In [ ]:
from finalproject.loss import from_grid_to_coordinates
import PIL.Image
from finalproject.utils import annotate_image
from matplotlib import pyplot as plt


prediction = preds[0]
prediction = from_grid_to_coordinates(prediction)
img = PIL.Image.open(file_path[0].numpy().decode('utf-8'))
img = annotate_image(img, targets=prediction[:, 0:4], scores=prediction[:, 4], labels=tf.constant([0]), normalized=True)
plt.figure(figsize=(4, 4), facecolor='black')
plt.imshow(img)
plt.axis('off')
plt.show()

In [2]:
import pandas as pd
import os
from PIL import Image

# ---- CONFIG ----
CSV_PATH = "banana-detection/bananas_val/label.csv"        # your csv file
IMAGES_DIR = "banana-detection/bananas_val/images"          # folder containing images
OUTPUT_DIR = "banana-detection/bananas_val/images/labels"     # where .txt files will be saved
# -----------------

os.makedirs(OUTPUT_DIR, exist_ok=True)

# Load CSV
df = pd.read_csv(CSV_PATH)

for img_name, group in df.groupby("img_name"):
    # Load image to get width/height
    img_path = os.path.join(IMAGES_DIR, img_name)
    img = Image.open(img_path)
    img_w, img_h = img.size

    yolo_lines = []

    for _, row in group.iterrows():
        cls = int(row["label"])
        xmin, ymin, xmax, ymax = row["xmin"], row["ymin"], row["xmax"], row["ymax"]

        # Convert to YOLO format
        x_center = (xmin + xmax) / 2.0 / img_w
        y_center = (ymin + ymax) / 2.0 / img_h
        width = (xmax - xmin) / img_w
        height = (ymax - ymin) / img_h

        yolo_lines.append(f"{cls} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}")

    # Write YOLO file
    label_filename = os.path.splitext(img_name)[0] + ".txt"
    out_path = os.path.join(OUTPUT_DIR, label_filename)

    with open(out_path, "w") as f:
        f.write("\n".join(yolo_lines))

print("Conversion complete!")

Conversion complete!
